In [3]:
import face_recognition
import os
import pickle

In [4]:
def generate_encodings(folder_path):

    all_files = os.listdir(folder_path)

    # Filter for image files (e.g., .jpg, .png, .jpeg)
    img_path = [filename for filename in all_files if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.gif', '.bmp'))]

    face_encodings = []
    face_name = []

    for i in img_path:
        picture_of_me = face_recognition.load_image_file(f"./photos/{i}")
        face_encodings.append(face_recognition.face_encodings(picture_of_me)[0])
        face_name.append(i.split('.')[0].strip())
        
        
    with open('face_encodings', 'wb') as file:
        pickle.dump(face_encodings, file)

    with open('people', 'wb') as file:
        pickle.dump(face_name, file)

generate_encodings("./photos")

In [6]:
def load_encondings(face_encodings_pkl_path: str, people_pkl_path: str) -> list:

    with open(face_encodings_pkl_path, 'rb') as file:
        face_encodings = pickle.load(file)
    
    with open(people_pkl_path, 'rb') as file:
        face_name = pickle.load(file)

    return face_encodings, face_name

load_encondings('./face_encodings', './people')

([], [])

In [7]:
import face_recognition
import cv2
import numpy as np

def load_encondings(face_encodings_pkl_path: str, people_pkl_path: str) -> list:

    with open(face_encodings_pkl_path, 'rb') as file:
        face_encodings = pickle.load(file)
    
    with open(people_pkl_path, 'rb') as file:
        face_name = pickle.load(file)

    return face_encodings, face_name



known_face_encodings, known_face_names = load_encondings('./face_encodings', './people')

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

TypeError: compute_face_descriptor(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], face: _dlib_pybind11.full_object_detection, num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vector
    2. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], num_jitters: int = 0) -> _dlib_pybind11.vector
    3. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], faces: _dlib_pybind11.full_object_detections, num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vectors
    4. (self: _dlib_pybind11.face_recognition_model_v1, batch_img: List[numpy.ndarray[(rows,cols,3),numpy.uint8]], batch_faces: List[_dlib_pybind11.full_object_detections], num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vectorss
    5. (self: _dlib_pybind11.face_recognition_model_v

In [ ]:
import mysql.connector
import pandas as pd

test = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI'
)

test_cursor = test.cursor()
test_cursor.execute("show databases;")
dbs = test_cursor.fetchall()

test_cursor.close()
test.close()

dbs

In [ ]:
db = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI',
    database='sql12644747'
)
mycursor = db.cursor()

mycursor.execute("""
show tables
""")
result = mycursor.fetchall()

mycursor.close()
db.close()

result

In [ ]:
db = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI',
    database='sql12644747'
)
mycursor = db.cursor()

mycursor.execute("""
describe attendence
""")
result = mycursor.fetchall()

mycursor.close()
db.close()

result

In [ ]:
db = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI',
    database='sql12644747'
)
mycursor = db.cursor()

mycursor.execute("""
INSERT INTO attendence (name, number)
VALUES ('Sudeep', 6969);
""")
db.commit()

mycursor.close()
db.close()

# result

In [ ]:
db = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI',
    database='sql12644747'
)
mycursor = db.cursor()

mycursor.execute("""
select * from attendence
""")
result = mycursor.fetchall()

mycursor.close()
db.close()

pd.DataFrame(result)